# 디스크립터 실전  
- 디스크립터를 통해 처리할 수 있는 몇 가지 상황 알아보기  
- 디스크립터에 대한 여러 구현 방법과 각각의 장단점 등 살펴보기
- 디스크립터를 사용하기에 가장 적합한 시나리오는 어떤 것들이 있는 지 알아보기  

## 디스크립터를 사용한 애플리케이션  
- 최종적으로 코드 중복을 최소화 하는 방향의 예제로 발전시키기 
- 중복 코드를 디스크립터로 추상화함으로써 클라이언트의 코드 줄이기

### 디스크립터를 사용하지 않은 예  
예제: 속성을 가진 일반 클래스의 속성 값이 달라질 때마다 추적  
-> 방법1: 속성의 setter 메서드에서 값이 변경될 때 검사하여 리스트와 같은 내부 변수에 값을 저장하기  
예제의 내용: 여행자 클래스, 현재 어느 도시에 있는 지를 속성으로 가짐. 사용자가 방문한 모든 도시를 추적

In [1]:
class Traveller:
    def __init__(self, name, current_city):
        self.name = name
        self._current_city = current_city
        self._cities_visited = [current_city]
        
    @property
    def current_city(self):
        return self._current_city
    
    @current_city.setter
    def current_city(self, new_city):
        if new_city != self._current_city:
            self._cities_visited.append(new_city)
            self._current_city = new_city
            
    @property
    def cities_visited(self):
        return self._cities_visited

In [4]:
alice = Traveller("Alice", "Barcelona")
alice.current_city = "Paris"
alice.current_city = "Brussels"
alice.current_city = "Amsterdam"
alice.cities_visited

['Barcelona', 'Paris', 'Brussels', 'Amsterdam']

위의 기능만 구현해도 된다면 추가로 구현할 필요 없이 프로퍼티만 사용해도 충분함.  
그러나 애플리케이션 여러 곳에서 똑같은 로직을 사용하여 속성의 모든 변수를 추적해야한다면 같은 로직을 반복해야함.  

### 이상적인 구현 방법  
모든 클래스에 적용할 수 있도록 디스크립터를 사용하여 이전 섹션의 문제를 해결해보기  
속성에 대해 이름을 가진 일반적인 디스크립터 생성. 이 디스크립터는 값이 달라질 경우 리스트에 저장하여 추적하는 기능을 가짐  

디스크립터는 특정 도메인과 관련이 없게 다른 유형의 클래스에서도 동일한 결과를 내도록 설계되었음

In [5]:
class HistoryTracedAttribute:
    def __init__(self, trace_attribute_name) -> None:
        """
        속성의 이름은 디스크립터에 할당된 변수 중 하나로 이 예제에서는 current_city임
        이에대한 추적을 저장할 변수의 이름을 디스크립터에 전달.
        이 예에서는 cities_visited라는 속성에 current_city의 모든 값을 추적하도록 지시
        """  
        self.trace_attribute_name = trace_attribute_name
        self._name = None
        
    def __set_name__(self, owner, name):
        self._name = name
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name] 
    
    def __set__(self, instance, value):
        self._track_change_in_value_for_instance(instance, value)
        instance.__dict__[self._name] = value
        
    def _track_change_in_value_for_instance(self, instance, value):
        """
        디스크립터를 처음으로 호출할 때는 추적 값이 존재하지 않을 것이므로 나중에 추가할 수 있도록 비어있는 배열로 초기화
        """
        self._set_default(instance)
        if self._needs_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value)
            
    def _needs_to_track_change(self, instance, value) -> bool:
        try:
            current_value = instance.__dict__[self._name]
        except KeyError:
            """
            처음 Traveller를 호출할 때는 방문지가 없으므로 인스턴스 사전에서 current_city 키도 존재하 않음
            이런 경우도 새로운 여행지가 생긴 것이므로 추적의 대상이 됨.
            """
            return true
        return value != current_value # 새 값이 설정된 값과 다를 때만 변경 사항 저장
    
    def _set_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, []) 
        # 사전의 setdefault 메서드는 KeyError를 피하기 위해서 사용됨
        # setdefault는 두 개의 파라미터를 받는데 첫 번째 파라미터의 키가 있으면 해당 값을 반환하고, 없으면 두번째 파라미터를 반환
        
class Traveller:
    current_city = HistoryTracedAttribute("cities_visited")
    
    def __init__(self, name, current_city):
        """
        Traveller의 __init__메서드에서 디스크립터가 이미 생성된 단계.
        할당 명령은 2단계 값을 추적하기 위한 빈 리스트 만들기를 실행하고, 
        3단계를 실행하여 리스트에 값을 추가하고 나중에 검색하기 위한 키를 설정함
        """
        self.name = name
        self.current_city = current_city

디스크립터 코드는 다소 복잡해졌으나 클라이언트 클래스의 코드는 상당히 간단해짐.  
위의 디스크립터를 여러번 사용한다면 디스크립터를 사용하는 것이 클래스마다 중복 코드를 생성하는 것보다 나음  
디스크립터 안에는 어떠한 비즈니스 로직도 포함되어 있지 않기 때문에 완전히 다른 어떤 클래스에서 적용하여도 같은 효과를 냄  
디스크립터는 비즈니스 로직의 구현보다는 라이브럴, 프레임워크 또는 내부 API를 정의하는 데 적합함.